# nb-model_xg-mdl-tcn-2

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from collections import OrderedDict
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg-mdl-tcn-2.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import torchfunc
import pytorch_lightning as pl

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MODEL_DIR, RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, is_type, pd_common_idx_rows, remove_dups_list, NestedDefaultDict, set_loglevel, search_df, chained_filter, get_variants, load_df, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import isnt, window_iter, np_assert_identical_len_dim, midx_get_level, pd_rows, midx_intersect, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_at_least_nd, np_is_ndim, identity_fn
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, ERROR_CODE, TEST_RATIO, VAL_RATIO, EXPECTED_NUM_HOURS, default_dataset
from model.common import PYTORCH_ACT_MAPPING, PYTORCH_OPT_MAPPING, PYTORCH_SCH_MAPPING, PYTORCH_LOSS_MAPPING
from model.xg_util import xgload
from model.preproc_util import temporal_preproc
from model.train_util import pd_get_np_tvt, batchify
from model.pl_util import TCNModel
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_xg-mdl-tcn-2.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Load Data

In [2]:
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']
chosen_asset = assets[0]

In [3]:
f = xgload(XG_DATA_DIR +'features' +sep)
l = xgload(XG_DATA_DIR +'labels' +sep)
t = xgload(XG_DATA_DIR +'targets' +sep)

In [4]:
print('num f: {}'.format(len(list(f))))
print('num l: {}'.format(len(list(l))))
print('num t: {}'.format(len(list(t))))

num f: 2520
num l: 1008
num t: 1504


### ddir / dret

In [5]:
ddir_pba_hoc = {a: list(l.childkeys([a, 'ddir', 'ddir', 'pba_hoc_hdxret_ddir'])) for a in assets}
ddir_vol_hoc = {a: list(l.childkeys([a, 'ddir', 'ddir', 'vol_hoc_hdxret_ddir'])) for a in assets}

In [6]:
dret_pba_hoc = {a: list(t.childkeys([a, 'dret', 'dret', 'pba_hoc_hdxret_dret'])) for a in assets}
dret_vol_hoc = {a: list(t.childkeys([a, 'dret', 'dret', 'vol_hoc_hdxret_dret'])) for a in assets}

### ddir1 / dret1

In [7]:
groups = ['lin', 'log']
fmt3, fmt4 = '{}_{}', '{}_hdxret1_{}'

In [8]:
e = 'ddir1'
b = 'pba_hoc'; ddir1_pba_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; ddir1_pba_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; ddir1_vol_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; ddir1_vol_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

In [9]:
e = 'dret1'
b = 'pba_hoc'; dret1_pba_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; dret1_pba_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; dret1_vol_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; dret1_vol_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

### ddir2/dret2

In [10]:
scalars = ['0.5', '1', '2']
stats = ['avg', 'std', 'mad', 'max', 'min']
fmt4, fmt5 = '{}_hdxret2_{}', '{}_hdxret2({}*{},1)_{}'

In [11]:
e = 'ddir2'
b = 'pba_hoc'; ddir2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; ddir2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; ddir2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; ddir2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

In [12]:
e = 'dret2'
b = 'pba_hoc'; dret2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; dret2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; dret2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; dret2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

### dxfbdir1 / dxfbret1

In [13]:
groups = ['lin', 'log']
fmt3, fmt4 = '{}_{}', '{}_hdxcret1_{}'

In [14]:
e = 'dxfbdir1'
b = 'pba_hoc'; dxfbdir1_pba_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; dxfbdir1_pba_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; dxfbdir1_vol_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; dxfbdir1_vol_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

In [15]:
e = 'dxfbcret1'
b = 'pba_hoc'; dxfbcret1_pba_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; dxfbcret1_pba_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; dxfbcret1_vol_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; dxfbcret1_vol_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

### dxfbdir2 / dxfbcret2

In [16]:
scalars = ['0.5', '1', '2']
stats = ['avg', 'std', 'mad', 'max', 'min']
fmt4, fmt5 = '{}_hdxcret2_{}', '{}_hdxcret2({}*{},1)_{}'

In [17]:
e = 'dxfbdir2'
b = 'pba_hoc'; dxfbdir2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; dxfbdir2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; dxfbdir2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; dxfbdir2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

In [18]:
e = 'dxfbcret2'
b = 'pba_hoc'; dxfbcret2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; dxfbcret2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; dxfbcret2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; dxfbcret2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

### Features

In [19]:
list(set([k[1] for k in f.childkeys([assets[0]])]))

['dwrxmx',
 'hdzn',
 'dwrod',
 'hdgau',
 'dohlca',
 'hdod',
 'dffd',
 'hohlca',
 'dwrzn',
 'dwrpt',
 'dwrmx',
 'hdpt',
 'hdmx',
 'ddiff',
 'hduni',
 'dlogret',
 'dc']

In [20]:
kc_end = ['ddiff', 'ddiff_pba_vol']
ft_all = {a: list(f.childkeys([a, *kc_end])) for a in assets}
feat = ft_all[chosen_asset]

In [21]:
feat

[['sp_500',
  'ddiff',
  'ddiff_pba_vol',
  'pba_dohlca_ddiff',
  'pba_dohlca_ddiff(1)'],
 ['sp_500',
  'ddiff',
  'ddiff_pba_vol',
  'vol_dohlca_ddiff',
  'vol_dohlca_ddiff(1)']]

## Select Data

In [22]:
features_df = inner_join(f[feat[0]], f[feat[1]])
no_zero = lambda df: df[df.values.sum(axis=1) != 0]
to_bin = lambda df: (df+1)*.5
feature_df, label_df, target_df = pd_common_idx_rows(features_df, to_bin(no_zero(l[ddir_pba_hoc[chosen_asset][0]])), t[dret_pba_hoc[chosen_asset][0]])
assert(feature_df.shape[0]==label_df.shape[0]==target_df.shape[0])

## Mdl

In [23]:
m_p = {
	'window_size': 10,
	'num_blocks': 1,
	#'block_channels': [[30, 20, 10]],
	'block_channels': [[5]],
	'block_act': 'elu',
	'out_act': 'relu',
	'kernel_sizes': [3],
	'dilation_index': 'global',
	'global_dropout': .2,
	'no_dropout': [0],
}
t_p = {
	'epochs': 200,
	'batch_size': 64,
	'loss': 'nll',    
	'opt': {
		'name': 'adam',
		'kwargs': {
			'lr': .001
		}
	},
	'sch': {
		'name': 'rpl',
		'kwargs': {
			'mode': 'min',
			'factor': 0.1,
			'patience': 10,
			'threshold': 0.0001,
			'threshold_mode': 'rel',
			'cooldown': 0,
			'min_lr': 0
		}
	}
}

In [24]:
m_p

{'window_size': 10,
 'num_blocks': 1,
 'block_channels': [[5]],
 'block_act': 'elu',
 'out_act': 'relu',
 'kernel_sizes': [3],
 'dilation_index': 'global',
 'global_dropout': 0.2,
 'no_dropout': [0]}

In [25]:
t_p

{'epochs': 200,
 'batch_size': 64,
 'loss': 'nll',
 'opt': {'name': 'adam', 'kwargs': {'lr': 0.001}},
 'sch': {'name': 'rpl',
  'kwargs': {'mode': 'min',
   'factor': 0.1,
   'patience': 10,
   'threshold': 0.0001,
   'threshold_mode': 'rel',
   'cooldown': 0,
   'min_lr': 0}}}

In [26]:
mdl = TCNModel(m_p, t_p, (feature_df, label_df, target_df))

In [27]:
for batch in mdl.train_dataloader():
    print(len(batch))
    x, y, z = batch
    print(x.shape)
    print(y.shape)
    break
    for b in batch:
        print(len(b))
        print(b.shape)
    break

3
torch.Size([64, 10, 10])
torch.Size([64])


In [28]:
print(x.shape, y.shape)
fmdl = mdl.float()
fmdl.forward(x.float())

ERROR:root:self.net(x).shape:  torch.Size([64, 5, 8])
ERROR:root:residual.shape: torch.Size([64, 5, 10])


torch.Size([64, 10, 10]) torch.Size([64])
The size of tensor a (8) must match the size of tensor b (10) at non-singleton dimension 2


SystemExit: 0

/home/kev/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [29]:
fmdl

TCNModel(
  (loss): NLLLoss()
  (clf): Classifier(
    (emb): TemporalConvNet(
      (convnet): Sequential(
        (RB_0): ResidualBlock(
          (net): Sequential(
            (TL_0_0): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(10, 5, kernel_size=(3,), stride=(1,))
                (1): ELU(alpha=1.0)
                (2): Dropout(p=0, inplace=False)
              )
            )
          )
          (downsample): Conv1d(10, 5, kernel_size=(1,), stride=(1,))
          (out_act): ReLU()
        )
      )
    )
    (out): Linear(in_features=5, out_features=1, bias=True)
    (logprob): LogSoftmax()
  )
)

In [30]:
%tb

SystemExit: 0

In [31]:
mdl

TCNModel(
  (loss): NLLLoss()
  (clf): Classifier(
    (emb): TemporalConvNet(
      (convnet): Sequential(
        (RB_0): ResidualBlock(
          (net): Sequential(
            (TL_0_0): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(10, 5, kernel_size=(3,), stride=(1,))
                (1): ELU(alpha=1.0)
                (2): Dropout(p=0, inplace=False)
              )
            )
          )
          (downsample): Conv1d(10, 5, kernel_size=(1,), stride=(1,))
          (out_act): ReLU()
        )
      )
    )
    (out): Linear(in_features=5, out_features=1, bias=True)
    (logprob): LogSoftmax()
  )
)

In [31]:
def tcn_setup_data(data):
        feature_df, label_df, target_df = data
        ftrain, fval, ftest = map(np_at_least_nd, pd_get_np_tvt(feature_df, as_midx=False))
        ltrain, lval, ltest = map(partial(np_at_least_nd, axis=-1), pd_get_np_tvt(label_df, as_midx=False))
        ttrain, tval, ttest = map(partial(np_at_least_nd, axis=-1), pd_get_np_tvt(target_df, as_midx=False))
        return ftrain, ltrain, ttrain

In [32]:
"""
d = temporal_preproc(tcn_setup_data((feature_df, label_df, target_df)), window_size=m_p['window_size'])
print(d[0].shape)
print(d[0][0:3])
print(d[1][0:3])
#print(d[0][0][0][:, :, :-4])
chomp_size = 10
print(x[:, :, :-chomp_size])
"""

"\nd = temporal_preproc(tcn_setup_data((feature_df, label_df, target_df)), window_size=m_p['window_size'])\nprint(d[0].shape)\nprint(d[0][0:3])\nprint(d[1][0:3])\n#print(d[0][0][0][:, :, :-4])\nchomp_size = 10\nprint(x[:, :, :-chomp_size])\n"

In [33]:
trainer = pl.Trainer(max_nb_epochs=t_p['epochs'], gpus=1, amp_level='O1', use_amp=True)
trainer.fit(mdl.float())

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


TypeError: forward() missing 1 required positional argument: 'target'